In [1]:
import numpy as np
import pandas as pd
import os

In [14]:
songs = pd.read_excel('../data/songs_short.xlsx')
user = pd.read_excel('../data/user.xlsx')
history = pd.read_excel('../data/listening history.xlsx')


In [16]:
print("Songs head:")
print(songs.head())

print("User head:")
print(user.head())

print("History head:")
print(history.head())

Songs head:
               spotify_id                          Song         Artist  Rank  \
0  4H2TRR9FjnnIwxGnIt9stO                  Killin' Time    Clint Black     1   
1  6iy4PoAuZBMvtrlDX4VxC7               Check Yes Or No  George Strait     2   
2  3oW6SWwGqiZSPTiAp7ZQoH  It's A Great Day To Be Alive   Travis Tritt     3   
3  7lUE02KHkZM44BZgjCaWRO            Meet in the Middle    Diamond Rio     4   
4  1ecIqo8pOc8S6y3l80iGxd                         Fancy  Reba McEntire     5   

     Genre  
0  country  
1  country  
2  country  
3  country  
4  country  
User head:
   ID  Password                 Name Genre 1  Genre 2  Genre 3
0   1         1           Sibyl Bode    rock      r&b      pop
1   2         2    Eleazar Hettinger     rap     soul  hip hop
2   3         3        Geovanny Lang     rap  country     soul
3   4         4    Micah Satterfield    jazz      r&b     soul
4   5         5  Courtney Schowalter    soul     folk    metal


In [17]:
from h2ogpte import H2OGPTE

In [11]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-ms7SU43E34tS9UJks5RD2KM3m1JumOR2pM73Dk95VzKjM6TZ',
)

# Create Collection
collection_id = client.create_collection(
    name="Musicrecco",
    description="Music Reccomendation Chatbot",
)

# Upload Files
songs_path = '../data/songs_short.xlsx'
user_path = '../data/user.xlsx'
history_path = '../data/listening history.xlsx'


with open(songs_path, "rb") as f:
    songs_id = client.upload(os.path.basename(songs_path),f)
client.ingest_uploads(collection_id, [songs_id])

with open(user_path, "rb") as f:
    user_id = client.upload(os.path.basename(user_path),f)
client.ingest_uploads(collection_id, [user_id])

with open(history_path, "rb") as f:
    history_id = client.upload(os.path.basename(history_path),f)
client.ingest_uploads(collection_id, [history_id])

Job(id='a4acb695-132b-46ea-bbdf-1bb013d5b433', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 28, 3, 10, 49, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='f6f52470dca241bbb96b6ff5d2649e4a', status='Indexing done.'), JobStatus(id='36fea923bf79437c86bd632a60267d92', status='Collecting done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 28, 3, 10, 53, tzinfo=TzInfo(UTC)), duration='4s', duration_seconds=4.0)

In [12]:
chat_session_id = client.create_chat_session(collection_id)
with client.connect(chat_session_id) as session:
    # Simple Question for Document Collection
        answer = session.query(
        message="what is the top id in users and top id in songs",
        rag_config={
            "rag_type": "rag",
        },
        ).content
        print(answer)


Based on the information provided, the top ID in users is 1 (Sibyl Bode) and the top ID in songs is not explicitly stated in the given text. However, the song with the earliest position in the list is "Killin' Time" by George Jones, which has a position of 1.


Prompt designs

In [ ]:
Prompt_template = """
Answer the questions based on only the following context:

{context}

---

Answer the questions based on the above context: {query}
"""
